# Compute EN vs. LN p-val maps

In [18]:
alphafdr = 0.1

skj_cp_tot_anom_enln_wrs_pval = wrspvalmap_loop(
    skj_cp_tot_anom[onienln==1],skj_cp_tot_anom[onienln==-1],'skj_cp_tot_anom_enln')
skj_cp_tot_anom_enln_wrs_ptf = controlfdr2d(skj_cp_tot_anom_enln_wrs_pval,alphafdr) 

bet_cp_tot_anom_enln_wrs_pval = wrspvalmap_loop(
    bet_cp_tot_anom[onienln==1],bet_cp_tot_anom[onienln==-1],'bet_cp_tot_anom_enln')
bet_cp_tot_anom_enln_wrs_ptf = controlfdr2d(bet_cp_tot_anom_enln_wrs_pval,alphafdr) 

bettoskj_cp_tot_anom_enln_wrs_pval = wrspvalmap_loop(
    bettoskj_cp_tot_anom[onienln==1],bettoskj_cp_tot_anom[onienln==-1],'bettoskj_cp_tot_anom_enln')
bettoskj_cp_tot_anom_enln_wrs_ptf = controlfdr2d(bettoskj_cp_tot_anom_enln_wrs_pval,alphafdr)

# Plot

In [ ]:
# - Define constant plot params
stipsizenow = 10
stipmarknow = 'o'
stipfacecolnow = 'none'
stipedgeltcolnow = 'lightgray'
stipedgedkcolnow = 'darkslategray'
stipewnow = 0.5 # marker edge width
cmnow = cm.PuOr
extendnow = 'both'

In [ ]:
# - Define subplot-variable plot params
stipltdkcosnow = 0.5*np.asarray(vmaxsnow) # light/dark stip cutoff value
mapsnow = [skj_cp_tot_anom[onienln==1].mean(dim='time'),
           bet_cp_tot_anom[onienln==1].mean(dim='time'),
           bettoskj_cp_tot_anom[onienln==1].mean(dim='time'),
           skj_cp_tot_anom[onienln==-1].mean(dim='time'),
           bet_cp_tot_anom[onienln==-1].mean(dim='time'),
           bettoskj_cp_tot_anom[onienln==-1].mean(dim='time')]
pvsnow = [skj_cp_tot_anom_enln_wrs_pval,
          bet_cp_tot_anom_enln_wrs_pval,
          bettoskj_cp_tot_anom_enln_wrs_pval,
          skj_cp_tot_anom_enln_wrs_pval,
          bet_cp_tot_anom_enln_wrs_pval,
          bettoskj_cp_tot_anom_enln_wrs_pval]
ptfsnow = [skj_cp_tot_anom_enln_wrs_ptf,
           bet_cp_tot_anom_enln_wrs_ptf,
           bettoskj_cp_tot_anom_enln_wrs_ptf,
           skj_cp_tot_anom_enln_wrs_ptf,
           bet_cp_tot_anom_enln_wrs_ptf,
           bettoskj_cp_tot_anom_enln_wrs_ptf]
#vminsnow = [-10, -1.6, -0.065, -10, -1.6, -0.065]
vminsnow = [-7, -1.5, -0.06, -7, -1.5, -0.06]
#vmaxsnow = [10, 1.6, 0.065, 10, 1.6, 0.065]
vmaxsnow = [7, 1.5, 0.06, 7, 1.5, 0.06]
titlesnow = ['SKJ CPUE - El Niño anom', 'BET CPUE - El Niño anom',
             'BET:SKJ CPUE - El Niño anom',
             'SKJ CPUE - La Niña anom', 'BET CPUE - La Niña anom',
             'BET:SKJ CPUE - La Niña anom']
unitsnow = ['[metric tons/set]', '[metric tons/set]', '',
            '[metric tons/set]', '[metric tons/set]', '']
signifstipsnow = [1,1,1,1,1,1]
ploteezsnow = [0,0,0,0,0,0]

In [ ]:
# - Set proj and define axes
fig,axes = plt.subplots(nrows=2, ncols=3, figsize=(16,7.5),
                        subplot_kw={'projection': ccrs.PlateCarree(central_longitude=200)})

# - Make maps pretty + plot
isp = 0
for irow in range(axes.shape[0]):
    for icol in range(axes.shape[1]):
        ax = axes[irow][icol]
        exec(open('helper_scripts/create_map_bgs.py').read())
        
        mapsnow[isp].plot(
            ax=ax, transform=ccrs.PlateCarree(), cmap=cmnow,
            vmin=vminsnow[isp], vmax=vmaxsnow[isp],
            cbar_kwargs={'pad': 0.02, 'label': unitsnow[isp],
                         'extend': extendnow})
        
        if ploteezsnow[isp]==1:
            nueezs.plot(ax=ax, transform=ccrs.PlateCarree(),
                color=eezfcnow, edgecolor=eezlcnow, linewidth=eezlwnow)
            
        if signifstipsnow[isp]==1:
            [ltcol_signiflonnow,ltcol_signiflatnow]=find_where_pval_small(
                pvsnow[isp].where(abs(mapsnow[isp])>stipltdkcosnow[isp]),
                ptfsnow[isp])
            [dkcol_signiflonnow,dkcol_signiflatnow]=find_where_pval_small(
                pvsnow[isp].where(abs(mapsnow[isp])<=stipltdkcosnow[isp]),
                ptfsnow[isp])
            ax.scatter(ltcol_signiflonnow, ltcol_signiflatnow,
                       marker=stipmarknow, linewidths=stipewnow,
                       facecolors=stipfacecolnow, edgecolors=stipedgeltcolnow,
                       s=stipsizenow, transform=ccrs.PlateCarree())
            ax.scatter(dkcol_signiflonnow, dkcol_signiflatnow,
                       marker=stipmarknow, linewidths=stipewnow,
                       facecolors=stipfacecolnow, edgecolors=stipedgedkcolnow,
                       s=stipsizenow, transform=ccrs.PlateCarree())
            
        ax.set_xlabel(''); ax.set_ylabel('')
        ax.set_title(titlesnow[isp])
        isp = isp + 1
       
fig.savefig(figpath + 'mainfig4_bscpue_ensomeans.png', bbox_inches='tight', pad_inches = 0, dpi = 200)